Details and Resources of the project in the doc:                                
https://docs.google.com/document/d/1F-KYQ5nRnDAUVDUrEbTFYGn7TEVsT8EByuM9xJLT4gA/edit

In [1]:
import warnings                        # To ignore any warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%pylab inline
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import seaborn as sns; sns.set()
import scipy.io as sio
# descriptive statistics
import scipy as sp
import scipy.stats as stats
import pywt

Populating the interactive namespace from numpy and matplotlib


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:

INPUT_DIR = '/content/drive/MyDrive/AI ML Things/University of Turku Research Internship'
'''
set_a_path=base_path+'/set_a'
set_a_metadata_path=base_path+'/set_a.csv'
set_b_path=base_path+'/set_b'
set_b_metadata_path=base_path+'/set_b.csv'
set_a_metadata = pd.read_csv(set_a_metadata_path)
'''
dataset_path = INPUT_DIR+'/set_a'
metadata = pd.read_csv(INPUT_DIR+'/set_a.csv')

SAMPLE_RATE = 16000
# seconds
MAX_SOUND_CLIP_DURATION=12  

#Explorer data

In [5]:
set_a=pd.read_csv(INPUT_DIR+"/set_a.csv")
set_a.head()

,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN


In [6]:
set_a_timing=pd.read_csv(INPUT_DIR+"/set_a_timing.csv")
set_a_timing.head()

,fname,cycle,sound,location
0,set_a/normal__201102081321.wav,1,S1,10021
1,set_a/normal__201102081321.wav,1,S2,20759
2,set_a/normal__201102081321.wav,2,S1,35075
3,set_a/normal__201102081321.wav,2,S2,47244
4,set_a/normal__201102081321.wav,3,S1,62992


In [7]:
set_b=pd.read_csv(INPUT_DIR+"/set_b.csv")
set_b.head()

,dataset,fname,label,sublabel
0,b,set_b/Btraining_extrastole_127_1306764300147_C...,extrastole,NaN
1,b,set_b/Btraining_extrastole_128_1306344005749_A...,extrastole,NaN
2,b,set_b/Btraining_extrastole_130_1306347376079_D...,extrastole,NaN
3,b,set_b/Btraining_extrastole_134_1306428161797_C...,extrastole,NaN
4,b,set_b/Btraining_extrastole_138_1306762146980_B...,extrastole,NaN


In [8]:
#merging both set-a and set-b
frames = [set_a, set_b]
train_ab=pd.concat(frames)
train_ab.describe()

,dataset,fname,label,sublabel
count,832,832,585,149
unique,2,832,5,2
top,b,set_b/Bunlabelledtest_176_1307988171173_A1.wav,normal,noisynormal
freq,656,1,351,120


Note: 'nan' indicate unclassified and unlabel test files

#Extracting Features of Data in Audio Domain

#Loading Data

murmur_noisymurmur_293_1311680805936_D.wav in set_B is corrupted removed it...

In [38]:
print("Number of training examples=", train_ab.shape[0], "  Number of classes=", len(train_ab.label.unique()))

Number of training examples= 832   Number of classes= 6


In [39]:
# get audio data with a fix padding may also chop off some file
def load_file_data (folder,file_names, duration=12, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    lables= []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file, sr=sr, duration=duration,res_type='kaiser_fast') 
            dur = librosa.get_duration(X, sr)

            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                y = librosa.util.fix_length(X, input_length)

            data.append(y)
            
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)        # , file



    return data

In [40]:
def dwt_features(data,no_dwt_features=48):
  
  arr=np.asarray(data) 
  waveletname = 'db1'

  ###############################
  """Extract The Coeeficients"""

  numcols =no_dwt_features     #Number of features extracted from Discrete wavelet transform here its 48
  numrows = arr.shape[0]

  Extracted_Features_tr=np.ndarray(shape=(numrows,numcols), dtype=float, order='F')

  for i in range(numrows):
    coeff = pywt.wavedec(arr[i,:], waveletname, level=6)
    cA6, cD6, cD5, cD4, cD3, cD2, cD1 = coeff
    Extracted_Features_tr[i, 0] = sp.mean(abs(cD1[:]))
    Extracted_Features_tr[i, 1] = sp.mean(abs(cD2[:]))
    Extracted_Features_tr[i, 2] = sp.mean(abs(cD3[:]))
    Extracted_Features_tr[i, 3] = sp.mean(abs(cD4[:]))
    Extracted_Features_tr[i, 4] = sp.mean(abs(cD5[:]))
    Extracted_Features_tr[i, 5] = sp.mean(abs(cD6[:]))
    Extracted_Features_tr[i, 6] = sp.mean(abs(cA6[:]))

    Extracted_Features_tr[i, 7] = sp.std(cD1[:])
    Extracted_Features_tr[i, 8] = sp.std(cD2[:])
    Extracted_Features_tr[i, 9] = sp.std(cD3[:])
    Extracted_Features_tr[i, 10] = sp.std(cD4[:])
    Extracted_Features_tr[i, 11] = sp.std(cD5[:])
    Extracted_Features_tr[i, 12] = sp.std(cD6[:])
    Extracted_Features_tr[i, 13] = sp.std(cA6[:])

    Extracted_Features_tr[i, 14] = stats.skew(cD1[:])
    Extracted_Features_tr[i, 15] = stats.skew(cD2[:])
    Extracted_Features_tr[i, 16] = stats.skew(cD3[:])
    Extracted_Features_tr[i, 17] = stats.skew(cD4[:])
    Extracted_Features_tr[i, 18] = stats.skew(cD5[:])
    Extracted_Features_tr[i, 19] = stats.skew(cD6[:])
    Extracted_Features_tr[i, 20] = stats.skew(cA6[:])

    Extracted_Features_tr[i, 21] = stats.kurtosis(cD1[:])
    Extracted_Features_tr[i, 22] = stats.kurtosis(cD2[:])
    Extracted_Features_tr[i, 23] = stats.kurtosis(cD3[:])
    Extracted_Features_tr[i, 24] = stats.kurtosis(cD4[:])
    Extracted_Features_tr[i, 25] = stats.kurtosis(cD5[:])
    Extracted_Features_tr[i, 26] = stats.kurtosis(cD6[:])
    Extracted_Features_tr[i, 27] = stats.kurtosis(cA6[:])

    Extracted_Features_tr[i, 28] = sp.median(cD1[:])
    Extracted_Features_tr[i, 29] = sp.median(cD2[:])
    Extracted_Features_tr[i, 30] = sp.median(cD3[:])
    Extracted_Features_tr[i, 31] = sp.median(cD4[:])
    Extracted_Features_tr[i, 32] = sp.median(cD5[:])
    Extracted_Features_tr[i, 33] = sp.median(cD6[:])
    Extracted_Features_tr[i, 34] = sp.median(cA6[:])

    Extracted_Features_tr[i, 35] = np.sqrt(
        np.mean(cD1[:] ** 2)); 
    Extracted_Features_tr[i, 36] = np.sqrt(
        np.mean(cD2[:] ** 2));
    Extracted_Features_tr[i, 37] = np.sqrt(
        np.mean(cD3[:] ** 2));
    Extracted_Features_tr[i, 38] = np.sqrt(
        np.mean(cD4[:] ** 2));
    Extracted_Features_tr[i, 39] = np.sqrt(
        np.mean(cD5[:] ** 2));
    Extracted_Features_tr[i, 40] = np.sqrt(
        np.mean(cD6[:] ** 2));
    Extracted_Features_tr[i, 41] = np.sqrt(
        np.mean(cA6[:] ** 2));

    Extracted_Features_tr[i, 42] = sp.mean(
        abs(cD1[:])) / sp.mean(abs(cD2[:]))
    Extracted_Features_tr[i, 43] = sp.mean(
        abs(cD2[:])) / sp.mean(abs(cD3[:]))
    Extracted_Features_tr[i, 44] = sp.mean(
        abs(cD3[:])) / sp.mean(abs(cD4[:]))
    Extracted_Features_tr[i, 45] = sp.mean(
        abs(cD4[:])) / sp.mean(abs(cD5[:]))
    Extracted_Features_tr[i, 46] = sp.mean(
        abs(cD5[:])) / sp.mean(abs(cD6[:]))
    Extracted_Features_tr[i, 47] = sp.mean(
        abs(cD6[:])) / sp.mean(abs(cA6[:]))

  return Extracted_Features_tr

In [41]:
'''
# load dataset-a, keep them separate for testing purpose
import os, fnmatch

A_folder=INPUT_DIR+'/set_a/'
# set-a
A_artifact_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'artifact*.wav')
A_artifact_sounds = dwt_features(load_file_data(folder=A_folder,file_names=A_artifact_files, duration=MAX_SOUND_CLIP_DURATION), 48)
A_artifact_labels = [1 for items in A_artifact_files]

A_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'normal*.wav')
A_normal_sounds = dwt_features(load_file_data(folder=A_folder,file_names=A_normal_files, duration=MAX_SOUND_CLIP_DURATION), 48)
A_normal_labels = [2 for items in A_normal_sounds]

A_extrahls_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'extrahls*.wav')
A_extrahls_sounds = dwt_features(load_file_data(folder=A_folder,file_names=A_extrahls_files, duration=MAX_SOUND_CLIP_DURATION), 48)
A_extrahls_labels = [3 for items in A_extrahls_sounds]

A_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'murmur*.wav')
A_murmur_sounds = dwt_features(load_file_data(folder=A_folder,file_names=A_murmur_files, duration=MAX_SOUND_CLIP_DURATION), 48)
A_murmur_labels = [4 for items in A_murmur_files]

# test files
A_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'Aunlabelledtest*.wav')
A_unlabelledtest_sounds = dwt_features(load_file_data(folder=A_folder,file_names=A_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION), 48)
A_unlabelledtest_labels = [-1 for items in A_unlabelledtest_sounds]

print ("loaded dataset-a with DWT features")





%%time
import os, fnmatch
# load dataset-b, keep them separate for testing purpose 
B_folder=INPUT_DIR+'/set_b_1/'
# set-b_1
B_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_1'), 'normal*.wav')  # include noisy files
B_normal_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_normal_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_normal_labels = [2 for items in B_normal_sounds]

B_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_1'), 'murmur*.wav')  # include noisy files
B_murmur_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_murmur_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_murmur_labels = [4 for items in B_murmur_files]

B_extrastole_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_1'), 'extrastole*.wav')
B_extrastole_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_extrastole_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_extrastole_labels = [5 for items in B_extrastole_files]

#test files
B_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_1'), 'Bunlabelledtest*.wav')
B_unlabelledtest_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_unlabelledtest_labels = [-1 for items in B_unlabelledtest_sounds]
print ("loaded dataset-b_1 with DWT features")





%%time
import os, fnmatch
# load dataset-b, keep them separate for testing purpose 
B_folder=INPUT_DIR+'/set_b_2/'
# set-b_2
B_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'normal*.wav')  # include noisy files
B_normal_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_normal_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_normal_labels = [2 for items in B_normal_sounds]

B_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'murmur*.wav')  # include noisy files
B_murmur_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_murmur_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_murmur_labels = [4 for items in B_murmur_files]

B_extrastole_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'extrastole*.wav')
B_extrastole_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_extrastole_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_extrastole_labels = [5 for items in B_extrastole_files]

#test files
B_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'Bunlabelledtest*.wav')
B_unlabelledtest_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_unlabelledtest_labels = [-1 for items in B_unlabelledtest_sounds]
print ("loaded dataset-b_2 with DWT features")

'''



import os, fnmatch
# load dataset-b_2, keep them separate for testing purpose 
B_folder=INPUT_DIR+'/set_b_2/'
# set-b_2
B_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'normal*.wav')  # include noisy files
B_normal_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_normal_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_normal_labels = [2 for items in B_normal_sounds]

B_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'murmur*.wav')  # include noisy files
B_murmur_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_murmur_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_murmur_labels = [4 for items in B_murmur_files]

B_extrastole_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'extrastole*.wav')
B_extrastole_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_extrastole_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_extrastole_labels = [5 for items in B_extrastole_files]

#test files
B_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b_2'), 'Bunlabelledtest*.wav')
B_unlabelledtest_sounds = dwt_features(load_file_data(folder=B_folder,file_names=B_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION), 48)
B_unlabelledtest_labels = [-1 for items in B_unlabelledtest_sounds]
print ("loaded dataset-b_2 with DWT features")


load file  /content/drive/MyDrive/AI ML Things/University of Turku Research Internship/set_b_2/normal__181_1308052613891_D.wav
fixing audio lenght : normal__181_1308052613891_D.wav
load file  /content/drive/MyDrive/AI ML Things/University of Turku Research Internship/set_b_2/normal__106_1306776721273_D1.wav
fixing audio lenght : normal__106_1306776721273_D1.wav
load file  /content/drive/MyDrive/AI ML Things/University of Turku Research Internship/set_b_2/normal__143_1306763822290_C.wav
fixing audio lenght : normal__143_1306763822290_C.wav
load file  /content/drive/MyDrive/AI ML Things/University of Turku Research Internship/set_b_2/normal_noisynormal_109_1305653972028_A.wav
fixing audio lenght : normal_noisynormal_109_1305653972028_A.wav
load file  /content/drive/MyDrive/AI ML Things/University of Turku Research Internship/set_b_2/normal__177_1307989650056_B.wav
fixing audio lenght : normal__177_1307989650056_B.wav
load file  /content/drive/MyDrive/AI ML Things/University of Turku Rese

In [ ]:
'''
#combine set-a 
x_data_a = np.concatenate((A_artifact_sounds, A_normal_sounds,A_extrahls_sounds,A_murmur_sounds))

y_data_a = np.concatenate((A_artifact_labels, A_normal_labels,A_extrahls_labels,A_murmur_labels))

test_x_a = np.copy((A_unlabelledtest_sounds))
test_y_a = np.copy((A_unlabelledtest_labels))

print ("combined training data record: ",len(y_data_a), len(test_y_a))
print ("combined training data record: ",len(x_data_a), len(test_x_a))


#combine set-b_1
x_data_b1 = np.concatenate((B_normal_sounds,B_murmur_sounds,B_extrastole_sounds))
 
y_data_b1 = np.concatenate((B_normal_labels,B_murmur_labels,B_extrastole_labels))
 
test_x_b1 = np.copy((B_unlabelledtest_sounds))
test_y_b1 = np.copy((B_unlabelledtest_labels))
 
print ("combined training data record: ",len(y_data_b1), len(test_y_b1))
print ("combined training data record: ",len(x_data_b1), len(test_x_b1))




#combine set-b_2 
x_data_b2 = np.concatenate((B_normal_sounds,B_murmur_sounds,B_extrastole_sounds))

y_data_b2 = np.concatenate((B_normal_labels,B_murmur_labels,B_extrastole_labels))

test_x_b2 = np.copy((B_unlabelledtest_sounds))
test_y_b2 = np.copy((B_unlabelledtest_labels))

print ("combined training data record: ",len(y_data_b2), len(test_y_b2))
print ("combined training data record: ",len(x_data_b2), len(test_x_b2))

'''


In [42]:
#combine set-b_2 
x_data_b2 = np.concatenate((B_normal_sounds,B_murmur_sounds,B_extrastole_sounds))

y_data_b2 = np.concatenate((B_normal_labels,B_murmur_labels,B_extrastole_labels))

test_x_b2 = np.copy((B_unlabelledtest_sounds))
test_y_b2 = np.copy((B_unlabelledtest_labels))

print ("combined training data record: ",len(y_data_b2), len(test_y_b2))
print ("combined training data record: ",len(x_data_b2), len(test_x_b2))

combined training data record:  327 0
combined training data record:  327 0


In [43]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "AI ML Things/University of Turku Research Internship/DWT_features/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/AI ML Things/University of Turku Research Internship/DWT_features/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [44]:
'''
np.save('x_data_a.npy',x_data_a)
np.save('y_data_a.npy',y_data_a)
np.save('test_x_a.npy',test_x_a)
np.save('test_y_a.npy',test_y_a)



np.save('x_data_b1.npy',x_data_b1)
np.save('y_data_b1.npy',y_data_b1)
np.save('test_x_b1.npy',test_x_b1)
np.save('test_y_b1.npy',test_y_b1)



np.save('x_data_b2.npy',x_data_b2)
np.save('y_data_b2.npy',y_data_b2)
np.save('test_x_b2.npy',test_x_b2)
np.save('test_y_b2.npy',test_y_b2)

'''

np.save('x_data_b2.npy',x_data_b2)
np.save('y_data_b2.npy',y_data_b2)
np.save('test_x_b2.npy',test_x_b2)
np.save('test_y_b2.npy',test_y_b2)



In [45]:
x_data_a=np.load('x_data_a.npy')
y_data_a=np.load('y_data_a.npy')
test_x_a=np.load('test_x_a.npy')
test_y_a=np.load('test_y_a.npy')


x_data_b1=np.load('x_data_b1.npy')
y_data_b1=np.load('y_data_b1.npy')
test_x_b1=np.load('test_x_b1.npy')
test_y_b1=np.load('test_y_b1.npy')

x_data_b2=np.load('x_data_b2.npy')
y_data_b2=np.load('y_data_b2.npy')
test_x_b2=np.load('test_x_b2.npy')
test_y_b2=np.load('test_y_b2.npy')

In [46]:
y_data_a.shape

(124,)

In [47]:
x_data_a.shape

(124, 48)

In [48]:
x_data_b1.shape

(133, 48)

In [49]:
y_data_b1.shape

(133,)

In [50]:
x_data_b2.shape

(327, 48)

In [51]:
y_data_b2.shape

(327,)

In [53]:
#combine set-a and set-b 
train_x = np.concatenate((x_data_a, x_data_b1, x_data_b2))

train_y = np.concatenate((y_data_a, y_data_b1, y_data_b2))

test_x = np.concatenate((test_x_a, test_x_b1, test_x_b2))
test_y = np.concatenate((test_y_a, test_y_b1, test_y_b2))

print ("combined training data record: ",len(train_y), len(test_y))

combined training data record:  584 246


In [54]:
np.save('train_x.npy',train_x)
np.save('train_y.npy',train_y)
np.save('test_x.npy',test_x)
np.save('test_y.npy',test_y)

In [56]:
x_data_wpd = np.concatenate((train_x, test_x))
y_data_wpd = np.concatenate((train_y, test_y))

In [57]:
x_data_wpd.shape

(830, 48)

In [58]:
y_data_wpd.shape

(830,)

In [59]:
np.save('x_data_dwt .npy',x_data_wpd)
np.save('y_data_dwt.npy',y_data_wpd)